In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pulse2percept.datasets import load_perezfornos2012
import time
from pulse2percept.models import TemporalModel
from pulse2percept.stimuli.pulse_trains import BiphasicPulseTrain
from scipy.optimize import minimize, differential_evolution
from datetime import datetime
import pandas as pd

In [ ]:
data = load_perezfornos2012()

In [ ]:
plt.plot(data['time_series'][0])

## train

In [ ]:
# fixed parameters - condition dependent
class Args():
    EAR = 20 # electrode activation rate 
    exper_dur = 30 # experiment duration
    stim_dur = 10 # stimulus duration
    interpolation_factor = 2 # number of frames sampled every second
    
args = Args()

In [ ]:
# fit parameters
PER=2 # persistence duration (usually between 0 and 8 seconds)
PFO=10  # perceptual fading onset (usually between 0 and 10 seconds)
PFD=10 # perceptual fading duration (usually between 0.5 and 60 seconds)
brightness_scale = 10

def brightness_equ(PER, PFO, PFD, brightness_scale):
    # fixed parameters
    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    # X0=24  # distance between two adjecent phosphenes in x axis
    # Y0=24  # distance between two adjecent phosphenes in y axis
    ElectrodeArraySize=1 # number of electrodes (60(val=1), 256(2), 961(3), or 2916(4) electrodes)
    ElectrodeArrayStructure=1 # electrode array structure (square(val = 1) or hexagonal(2)).

    va = 0
    vb = 0.333
    vc = 0.667
    vd = 1
    # MMr = 6  # number of rows
    # MMc = 10  # number of columns;
    # C = 240  # number of pixel columns
    # R = 144 # number of pixel rows, choose: phosphene size 0.4; X0 and Y0 = 24

    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    MMr = 3  # number of rows
    MMc = 3  # number of columns;
    X0 = 24  # distance between two adjecent phosphenes in x axis
    Y0 = 24  # distance between two adjecent phosphenes in y axis
    C = MMc*Y0  # number of pixel columns
    R = MMr*X0  # number of pixel rows, choose: phosphene size 0.4; X0 and Y0 = 24

    DFR = 30 # display frame rate (usually 30 or 60 FPS)

    v2 = 0
    v3 = 0
    count = 0

    EAR = args.EAR  # electrode activation rate (usually 6 or 10 Hz)
    exper_dur = args.exper_dur  # experiment duration
    stim_dur = args.stim_dur  # stimulus duration

    PER_Factor = 3.9/(PER*DFR) # 3.9 was set for 2 precent threshold, i.e. the phosphene falls to 2 precent of its original intensity;
    PF_Factor = PFO*EAR
    PFD_Factor = 3.9/(PFD*DFR)

    # Creating meshgrid
    x = np.arange(0, C) 
    y = np.arange(0, R)  
    X, Y = np.meshgrid(x, y)
    # Initialization
    z = np.zeros((R, C))
    Frame = np.zeros((R, C))
    FormerFrame = np.zeros((R, C))
    PFC = np.zeros((MMr, MMc))
    Theta = np.zeros((MMr, MMc))
    a = np.zeros((MMr, MMc))
    b = np.zeros((MMr, MMc))
    c = np.zeros((MMr, MMc))
    sigma_x = np.zeros((MMr, MMc))
    sigma_y = np.zeros((MMr, MMc))

    count = 0
    for s in range(MMr):
        for t in range(MMc):
            Theta[s, t] = 0
            sigma_x[s, t] = SigmaX
            sigma_y[s, t] = SigmaY
            a[s, t] = (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)
            b[s, t] = -(np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_y[s, t]**2)
            c[s, t] = (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)

    I_frame = [1]*(args.stim_dur*args.EAR)*MMr*MMc  # the brightness of the video is always 1
    # video frame rate is the same as the electrode firing rate
    
    I_frame_0 = [0]*(args.exper_dur*args.EAR-args.stim_dur*args.EAR)*MMr*MMc  # no visual input after stim_dur
    I_frame = np.append(np.array(I_frame), np.array(I_frame_0))
    I_frame = np.array(I_frame).reshape(args.exper_dur*args.EAR, MMr, MMc)

    results = []

    for frame in range(exper_dur*args.EAR):
        I = I_frame[frame]
        # I = I / I.max()
        MeanMatrix = np.zeros((MMr, MMc))
        s = 0
        t = 0
        v = 0

        start = int(np.floor((I.shape[0] % MMr) / 2) + np.floor(I.shape[0] / MMr) * MMr) - 1
        stop = int(1 + np.floor((I.shape[0] % MMr) / 2)) - 1
        step = -int(np.floor(I.shape[0] / MMr))
        for m in range(start, stop, step):
            for n in range(0, I.shape[1]//MMc*MMc - 1, I.shape[1] // MMc):
                for j in range(m, m - (I.shape[0] // MMr), -1):
                    # for k in range(n, n + (I.shape[1] // MMc)-2):
                    for k in range(n, n - (I.shape[1] // MMc), -1):
                        v += I[j, k]
                v /= ((I.shape[0] // MMr) * (I.shape[1] // MMc))

                if 0 <= v <= 0.25:
                    v = va
                elif 0.25 < v <= 0.5:
                    v = vb
                elif 0.5 < v <= 0.75:
                    v = vc
                else:
                    v = vd
                MeanMatrix[s, t] = v
                v = 0
                t += 1
            t = 0
            s += 1

        getting_all_subframe = []

        SubFrame_range = int(DFR) // int(EAR) + 1
        if SubFrame_range <= 2: SubFrame_range = 2
        for SubFrame in range(1, SubFrame_range):
            for s in range(MMr):
                for t in range(MMc):
                    adjusted_t = t + 1
                    adjusted_s = s + 1
                    z += MeanMatrix[s, t] * np.exp(
                        # -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        # (a[s, t] * (X - X0 * t - v2 + X0 // 2) ** 2 +
                        (a[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) ** 2 +
                        # 2 * b[s, t] * (X - X0 * t - v2 + X0 // 2) * (Y - Y0 * s + Y0 // 2) +
                        2 * b[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) * (Y - Y0 * adjusted_s + Y0 // 2) +
                        # c[s, t] * (Y - Y0 * s + Y0 // 2) ** 2))
                        c[s, t] * (Y - Y0 * adjusted_s + Y0 // 2) ** 2))

                v2 -= (X0 / 2) * v3
                v3 *= -1

            for s in range(R, 0, -X0):
                for t in range(0, C, Y0):
                    if SubFrame == 1:
                        condition1 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) == 0 and np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) == 0)
                        condition2 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) < np.mean(FormerFrame[s-X0:s-1, t:t+Y0-2]))
                        condition3 = (MeanMatrix[int(s/X0)-1, t // Y0] == 0)
                        condition4 = (PFC[int(s/X0)-1, t // Y0] < PF_Factor)
                        if condition1 or condition2 or condition3 and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                        elif np.mean(z[s-X0:s-1, t-1:t+Y0-2]) >= np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = z[s-X0:s-1, t-1:t+Y0-2]
                            PFC[s//X0-1, t//Y0] += 1
                        elif PFC[s//X0-1, t//Y0] >= PF_Factor:
                            # Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-3 * PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PFD_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                    else:
                        Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * Frame[s-X0:s-1, t-1:t+Y0-2]
            z = np.zeros((R,C))
            count = count + 1

            FormerFrame = Frame
            getting_all_subframe.append(Frame.max())
        results.append(max(getting_all_subframe))

    output =  [x * brightness_scale for x in results]

    output_ = []
    combined_frame = args.EAR/args.interpolation_factor
    for i in range(args.exper_dur*args.interpolation_factor):
        output_.append(max(output[int(i*combined_frame):int((i+1)*combined_frame)]))
    return output_


In [ ]:
import numpy as np
from scipy.optimize import minimize

def objective_function(params, gt_curves_train):
    # persistence time, fading time onset, fading time duration, brightness scale
    PER, PFO, PFD, brightness_scale = params
    
    total_mse = 0
    y_predicted = brightness_equ(PER, PFO, PFD, brightness_scale)
    for y in gt_curves_train: total_mse += np.mean((np.array(y_predicted) - y) ** 2)
        
    print(total_mse/len(gt_curves_train))
    
    return total_mse/len(gt_curves_train)


for cond in range(5):
    for subj in range(9):
        current_time = datetime.now()
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

        i = cond*9 + subj

        if i < -1 : pass
        else:
            idx_lst = list(range(cond*9 , (cond+1)*9))
            idx_lst.remove(i)
            
            if i <= 8: 
                args.EAR = 20
                args.exper_dur = 30
                args.stim_dur = 10
            elif i <= 17: 
                args.EAR = 5
                args.exper_dur = 30
                args.stim_dur = 10
            elif i <= 26:
                args.EAR = 60
                args.exper_dur = 30
                args.stim_dur = 10
                
            elif i<=35: 
                args.EAR = 20
                args.exper_dur = 10
                args.stim_dur = 1
            else: 
                args.EAR = 20
                args.exper_dur = 70
                args.stim_dur = 60
                
            # curve = np.nan_to_num(data['time_series'][i], 0)[4:args.exper_dur+4]
            # ground_truth = curve  
            
            gt_curves_train = [np.nan_to_num(data['time_series'][idx], 0)[4:args.exper_dur*args.interpolation_factor+4] for idx in idx_lst]
            gt_curves_val = np.nan_to_num(data['time_series'][i], 0)[4:args.exper_dur*args.interpolation_factor+4]
    
            # Initial guesses for parameters
            initial_params = [2, 10, 10, 10]
            # Run the optimization, passing the ground truth as an argument
            result = minimize(objective_function, initial_params, args=(gt_curves_train,), method='Nelder-Mead')
            # Extract optimized parameters
            optimized_params = result.x
            print("Optimized Parameters:", optimized_params)
            
            file_path = '/hdd/yuchen/baseline_test.txt'  
            line_to_add = '' 
            line_to_add += str(i) + ': '
            line_to_add += str(list(optimized_params)) + '\n' + formatted_time + '\n'
            with open(file_path, 'a') as file:
                file.write(line_to_add)

## train per panel 0.25 interp

In [ ]:
# fixed parameters - condition dependent
class Args():
    EAR = 20 # electrode activation rate 
    exper_dur = 30 # experiment duration
    stim_dur = 10 # stimulus duration
    interpolation_factor = 4 # number of frames sampled every second
    
args = Args()

In [ ]:
# fit parameters
PER=2 # persistence duration (usually between 0 and 8 seconds)
PFO=10  # perceptual fading onset (usually between 0 and 10 seconds)
PFD=10 # perceptual fading duration (usually between 0.5 and 60 seconds)
brightness_scale = 10

def brightness_equ(PER, PFO, PFD, brightness_scale):
    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    ElectrodeArraySize=1 # number of electrodes (60(val=1), 256(2), 961(3), or 2916(4) electrodes)
    ElectrodeArrayStructure=1 # electrode array structure (square(val = 1) or hexagonal(2)).

    va = 0
    vb = 0.333
    vc = 0.667
    vd = 1

    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    MMr = 2  # number of rows
    MMc = 3  # number of columns;
    X0 = 20  # distance between two adjecent phosphenes in x axis
    Y0 = 20  # distance between two adjecent phosphenes in y axis
    C = MMc*Y0  # number of pixel columns
    R = MMr*X0  # number of pixel rows, choose: phosphene size 0.4; X0 and Y0 = 24

    DFR = 30 # display frame rate (usually 30 or 60 FPS)

    v2 = 0
    v3 = 0
    count = 0

    EAR = args.EAR  # electrode activation rate (usually 6 or 10 Hz)
    exper_dur = args.exper_dur  # experiment duration
    stim_dur = args.stim_dur  # stimulus duration

    PER_Factor = 3.9/(PER*DFR) # 3.9 was set for 2 precent threshold, i.e. the phosphene falls to 2 precent of its original intensity;
    PF_Factor = PFO*EAR
    PFD_Factor = 3.9/(PFD*DFR)

    # Creating meshgrid
    x = np.arange(0, C) 
    y = np.arange(0, R)  
    X, Y = np.meshgrid(x, y)
    # Initialization
    z = np.zeros((R, C))
    Frame = np.zeros((R, C))
    FormerFrame = np.zeros((R, C))
    PFC = np.zeros((MMr, MMc))
    Theta = np.zeros((MMr, MMc))
    a = np.zeros((MMr, MMc))
    b = np.zeros((MMr, MMc))
    c = np.zeros((MMr, MMc))
    sigma_x = np.zeros((MMr, MMc))
    sigma_y = np.zeros((MMr, MMc))

    count = 0
    for s in range(MMr):
        for t in range(MMc):
            Theta[s, t] = 0
            sigma_x[s, t] = SigmaX
            sigma_y[s, t] = SigmaY
            a[s, t] = (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)
            b[s, t] = -(np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_y[s, t]**2)
            c[s, t] = (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)

    I_frame = [1]*(args.stim_dur*args.EAR)*MMr*MMc  # the brightness of the video is always 1
    # video frame rate is the same as the electrode firing rate
    
    I_frame_0 = [0]*(args.exper_dur*args.EAR-args.stim_dur*args.EAR)*MMr*MMc  # no visual input after stim_dur
    I_frame = np.append(np.array(I_frame), np.array(I_frame_0))
    I_frame = np.array(I_frame).reshape(args.exper_dur*args.EAR, MMr, MMc)

    results = []

    for frame in range(exper_dur*args.EAR):
        I = I_frame[frame]
        # I = I / I.max()
        MeanMatrix = np.zeros((MMr, MMc))
        s = 0
        t = 0
        v = 0

        start = int(np.floor((I.shape[0] % MMr) / 2) + np.floor(I.shape[0] / MMr) * MMr) - 1
        stop = int(1 + np.floor((I.shape[0] % MMr) / 2)) - 1
        step = -int(np.floor(I.shape[0] / MMr))
        for m in range(start, stop, step):
            for n in range(0, I.shape[1]//MMc*MMc - 1, I.shape[1] // MMc):
                for j in range(m, m - (I.shape[0] // MMr), -1):
                    # for k in range(n, n + (I.shape[1] // MMc)-2):
                    for k in range(n, n - (I.shape[1] // MMc), -1):
                        v += I[j, k]
                v /= ((I.shape[0] // MMr) * (I.shape[1] // MMc))

                if 0 <= v <= 0.25:
                    v = va
                elif 0.25 < v <= 0.5:
                    v = vb
                elif 0.5 < v <= 0.75:
                    v = vc
                else:
                    v = vd
                MeanMatrix[s, t] = v
                v = 0
                t += 1
            t = 0
            s += 1

        getting_all_subframe = []

        SubFrame_range = int(DFR) // int(EAR) + 1
        if SubFrame_range <= 2: SubFrame_range = 2
        for SubFrame in range(1, SubFrame_range):
            for s in range(MMr):
                for t in range(MMc):
                    adjusted_t = t + 1
                    adjusted_s = s + 1
                    z += MeanMatrix[s, t] * np.exp(
                        # -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        # (a[s, t] * (X - X0 * t - v2 + X0 // 2) ** 2 +
                        (a[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) ** 2 +
                        # 2 * b[s, t] * (X - X0 * t - v2 + X0 // 2) * (Y - Y0 * s + Y0 // 2) +
                        2 * b[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) * (Y - Y0 * adjusted_s + Y0 // 2) +
                        # c[s, t] * (Y - Y0 * s + Y0 // 2) ** 2))
                        c[s, t] * (Y - Y0 * adjusted_s + Y0 // 2) ** 2))

                v2 -= (X0 / 2) * v3
                v3 *= -1

            for s in range(R, 0, -X0):
                for t in range(0, C, Y0):
                    if SubFrame == 1:
                        condition1 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) == 0 and np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) == 0)
                        condition2 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) < np.mean(FormerFrame[s-X0:s-1, t:t+Y0-2]))
                        condition3 = (MeanMatrix[int(s/X0)-1, t // Y0] == 0)
                        condition4 = (PFC[int(s/X0)-1, t // Y0] < PF_Factor)
                        if condition1 or condition2 or condition3 and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                        elif np.mean(z[s-X0:s-1, t-1:t+Y0-2]) >= np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = z[s-X0:s-1, t-1:t+Y0-2]
                            PFC[s//X0-1, t//Y0] += 1
                        elif PFC[s//X0-1, t//Y0] >= PF_Factor:
                            # Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-3 * PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PFD_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                    else:
                        Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * Frame[s-X0:s-1, t-1:t+Y0-2]
            z = np.zeros((R,C))
            count = count + 1

            FormerFrame = Frame
            getting_all_subframe.append(Frame.max())
        results.append(max(getting_all_subframe))

    output =  [x * brightness_scale for x in results]

    output_ = []
    combined_frame = args.EAR/args.interpolation_factor
    for i in range(args.exper_dur*args.interpolation_factor):
        output_.append(max(output[int(i*combined_frame):int((i+1)*combined_frame)]))
    return output_


In [ ]:
import pandas as pd
data = pd.read_csv('/hdd/yuchen/perez-fornos-2012.csv')  

In [ ]:
import numpy as np
from scipy.optimize import minimize

args.interpolation_factor = 4

def objective_function(params, gt_curves_val):
    # persistence time, fading time onset, fading time duration, brightness scale
    PER, PFO, PFD, brightness_scale = params
    
    total_mse = 0
    y_predicted = brightness_equ(PER, PFO, PFD, brightness_scale)
    mse = np.mean((np.array(y_predicted) - gt_curves_val) ** 2)
    print(mse)
    return mse


for cond in range(5):
    for subj in range(9):
        current_time = datetime.now()
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

        i = cond*9 + subj

        if i < -1 : pass
        else:
            idx_lst = list(range(cond*9 , (cond+1)*9))
            idx_lst.remove(i)
            
            if i <= 8: 
                args.EAR = 20
                args.exper_dur = 30
                args.stim_dur = 10
            elif i <= 17: 
                args.EAR = 5
                args.exper_dur = 30
                args.stim_dur = 10
            elif i <= 26:
                args.EAR = 60
                args.exper_dur = 30
                args.stim_dur = 10
                
            elif i<=35: 
                args.EAR = 20
                args.exper_dur = 10
                args.stim_dur = 1
            else: 
                args.EAR = 20
                args.exper_dur = 70
                args.stim_dur = 60
                
            # gt_curves_val = np.nan_to_num(data['time_series'][i], 0)[4:args.exper_dur*args.interpolation_factor+4]
            gt_curves_val = np.nan_to_num(data.iloc[i, 6:args.exper_dur*args.interpolation_factor+6].to_numpy().astype('float64'), 0)  
            
    
            # Initial guesses for parameters
            initial_params = [2, 10, 10, 10]
            # Run the optimization, passing the ground truth as an argument
            result = minimize(objective_function, initial_params, args=(gt_curves_val,), method='Nelder-Mead')
            # Extract optimized parameters
            optimized_params = result.x
            print("Optimized Parameters:", optimized_params)
            
            file_path = '/hdd/yuchen/baseline_perpanel-025interp.txt'  
            line_to_add = '' 
            line_to_add += str(i) + ': '
            line_to_add += str(list(optimized_params)) + '\n' + formatted_time + '\n'
            with open(file_path, 'a') as file:
                file.write(line_to_add)

## train per-condition 0.25 interp

In [ ]:
# fixed parameters - condition dependent
class Args():
    EAR = 20 # electrode activation rate 
    exper_dur = 30 # experiment duration
    stim_dur = 10 # stimulus duration
    interpolation_factor = 2 # number of frames sampled every second
    
args = Args()

In [ ]:
import pandas as pd
data = pd.read_csv('perez-fornos-2012.csv')  

In [ ]:
# fit parameters
PER=2 # persistence duration (usually between 0 and 8 seconds)
PFO=10  # perceptual fading onset (usually between 0 and 10 seconds)
PFD=10 # perceptual fading duration (usually between 0.5 and 60 seconds)
brightness_scale = 10

def brightness_equ(PER, PFO, PFD, brightness_scale):
    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    ElectrodeArraySize=1 # number of electrodes (60(val=1), 256(2), 961(3), or 2916(4) electrodes)
    ElectrodeArrayStructure=1 # electrode array structure (square(val = 1) or hexagonal(2)).

    va = 0
    vb = 0.333
    vc = 0.667
    vd = 1

    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    MMr = 2  # number of rows
    MMc = 3  # number of columns;
    X0 = 20  # distance between two adjecent phosphenes in x axis
    Y0 = 20  # distance between two adjecent phosphenes in y axis
    C = MMc*Y0  # number of pixel columns
    R = MMr*X0  # number of pixel rows, choose: phosphene size 0.4; X0 and Y0 = 24

    DFR = 30 # display frame rate (usually 30 or 60 FPS)

    v2 = 0
    v3 = 0
    count = 0

    EAR = args.EAR  # electrode activation rate (usually 6 or 10 Hz)
    exper_dur = args.exper_dur  # experiment duration
    stim_dur = args.stim_dur  # stimulus duration

    PER_Factor = 3.9/(PER*DFR) # 3.9 was set for 2 precent threshold, i.e. the phosphene falls to 2 precent of its original intensity;
    PF_Factor = PFO*EAR
    PFD_Factor = 3.9/(PFD*DFR)

    # Creating meshgrid
    x = np.arange(0, C) 
    y = np.arange(0, R)  
    X, Y = np.meshgrid(x, y)
    # Initialization
    z = np.zeros((R, C))
    Frame = np.zeros((R, C))
    FormerFrame = np.zeros((R, C))
    PFC = np.zeros((MMr, MMc))
    Theta = np.zeros((MMr, MMc))
    a = np.zeros((MMr, MMc))
    b = np.zeros((MMr, MMc))
    c = np.zeros((MMr, MMc))
    sigma_x = np.zeros((MMr, MMc))
    sigma_y = np.zeros((MMr, MMc))

    count = 0
    for s in range(MMr):
        for t in range(MMc):
            Theta[s, t] = 0
            sigma_x[s, t] = SigmaX
            sigma_y[s, t] = SigmaY
            a[s, t] = (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)
            b[s, t] = -(np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_y[s, t]**2)
            c[s, t] = (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)

    I_frame = [1]*(args.stim_dur*args.EAR)*MMr*MMc  # the brightness of the video is always 1
    # video frame rate is the same as the electrode firing rate
    
    I_frame_0 = [0]*(args.exper_dur*args.EAR-args.stim_dur*args.EAR)*MMr*MMc  # no visual input after stim_dur
    I_frame = np.append(np.array(I_frame), np.array(I_frame_0))
    I_frame = np.array(I_frame).reshape(args.exper_dur*args.EAR, MMr, MMc)

    results = []

    for frame in range(exper_dur*args.EAR):
        I = I_frame[frame]
        # I = I / I.max()
        MeanMatrix = np.zeros((MMr, MMc))
        s = 0
        t = 0
        v = 0

        start = int(np.floor((I.shape[0] % MMr) / 2) + np.floor(I.shape[0] / MMr) * MMr) - 1
        stop = int(1 + np.floor((I.shape[0] % MMr) / 2)) - 1
        step = -int(np.floor(I.shape[0] / MMr))
        for m in range(start, stop, step):
            for n in range(0, I.shape[1]//MMc*MMc - 1, I.shape[1] // MMc):
                for j in range(m, m - (I.shape[0] // MMr), -1):
                    # for k in range(n, n + (I.shape[1] // MMc)-2):
                    for k in range(n, n - (I.shape[1] // MMc), -1):
                        v += I[j, k]
                v /= ((I.shape[0] // MMr) * (I.shape[1] // MMc))

                if 0 <= v <= 0.25:
                    v = va
                elif 0.25 < v <= 0.5:
                    v = vb
                elif 0.5 < v <= 0.75:
                    v = vc
                else:
                    v = vd
                MeanMatrix[s, t] = v
                v = 0
                t += 1
            t = 0
            s += 1

        getting_all_subframe = []

        SubFrame_range = int(DFR) // int(EAR) + 1
        if SubFrame_range <= 2: SubFrame_range = 2
        for SubFrame in range(1, SubFrame_range):
            for s in range(MMr):
                for t in range(MMc):
                    adjusted_t = t + 1
                    adjusted_s = s + 1
                    z += MeanMatrix[s, t] * np.exp(
                        # -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        # (a[s, t] * (X - X0 * t - v2 + X0 // 2) ** 2 +
                        (a[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) ** 2 +
                        # 2 * b[s, t] * (X - X0 * t - v2 + X0 // 2) * (Y - Y0 * s + Y0 // 2) +
                        2 * b[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) * (Y - Y0 * adjusted_s + Y0 // 2) +
                        # c[s, t] * (Y - Y0 * s + Y0 // 2) ** 2))
                        c[s, t] * (Y - Y0 * adjusted_s + Y0 // 2) ** 2))

                v2 -= (X0 / 2) * v3
                v3 *= -1

            for s in range(R, 0, -X0):
                for t in range(0, C, Y0):
                    if SubFrame == 1:
                        condition1 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) == 0 and np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) == 0)
                        condition2 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) < np.mean(FormerFrame[s-X0:s-1, t:t+Y0-2]))
                        condition3 = (MeanMatrix[int(s/X0)-1, t // Y0] == 0)
                        condition4 = (PFC[int(s/X0)-1, t // Y0] < PF_Factor)
                        if condition1 or condition2 or condition3 and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                        elif np.mean(z[s-X0:s-1, t-1:t+Y0-2]) >= np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = z[s-X0:s-1, t-1:t+Y0-2]
                            PFC[s//X0-1, t//Y0] += 1
                        elif PFC[s//X0-1, t//Y0] >= PF_Factor:
                            # Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-3 * PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PFD_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                    else:
                        Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * Frame[s-X0:s-1, t-1:t+Y0-2]
            z = np.zeros((R,C))
            count = count + 1

            FormerFrame = Frame
            getting_all_subframe.append(Frame.max())
        results.append(max(getting_all_subframe))

    output =  [x * brightness_scale for x in results]

    output_ = []
    combined_frame = args.EAR/args.interpolation_factor
    for i in range(args.exper_dur*args.interpolation_factor):
        output_.append(max(output[int(i*combined_frame):int((i+1)*combined_frame)]))
    return output_

In [ ]:
stim_info = {}
# EAR, exper_dur, stim_dur
for i in range(9): stim_info[i] = [20, 30, 10]
for i in range(9,18): stim_info[i] = [5, 30, 10]
for i in range(18,27): stim_info[i] = [60, 30, 10]
for i in range(27,36): stim_info[i] = [20, 10, 1]
for i in range(36,45): stim_info[i] = [20, 70, 60]

In [ ]:
import numpy as np
from scipy.optimize import minimize

args.interpolation_factor = 4

def objective_function(params, gt_curves_train, stim_lst):
    # persistence time, fading time onset, fading time duration, brightness scale
    PER, PFO, PFD, brightness_scale = params
    
    total_mse = 0
    for gt_idx in range(len(gt_curves_train)):
        gt_y = gt_curves_train[gt_idx]
        stim = stim_lst[gt_idx]

        args.EAR = stim[0]
        args.exper_dur = stim[1]
        args.stim_dur = stim[2]

        y_predicted = brightness_equ(PER, PFO, PFD, brightness_scale)
        total_mse += np.mean((np.array(y_predicted) - gt_y) ** 2)
    # print(total_mse/4)
    return total_mse/4

for subj in range(0, 9):
    for cond in range(5):

        i = cond*9 + subj
        print(i)

        if i != 22 : pass
        else:
            idx_lst = [0*9+subj, 1*9+subj, 2*9+subj, 3*9+subj, 4*9+subj]
            idx_lst.remove(i)
            
            args.EAR = stim_info[i][0]
            args.exper_dur = stim_info[i][1]
            args.stim_dur = stim_info[i][2]

            gt_curves_train = [np.nan_to_num(data.iloc[idx, 6:stim_info[idx][1]*args.interpolation_factor+6].to_numpy().astype('float64'), 0) for idx in idx_lst] 
            gt_curves_val = np.nan_to_num(data.iloc[i, 6:args.exper_dur*args.interpolation_factor+6].to_numpy().astype('float64'), 0)  

            stim_lst = [stim_info[idx] for idx in idx_lst]
            
            # Initial guesses for parameters
            initial_params = [2, 10, 10, 10]
            # Run the optimization, passing the ground truth as an argument
            result = minimize(objective_function, initial_params, args=(gt_curves_train,stim_lst), method='Nelder-Mead')
            # Extract optimized parameters
            optimized_params = result.x
            print("Optimized Parameters:", optimized_params)

            current_time = datetime.now()
            formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
        
            
            file_path = 'baseline_percondition_025interp.txt'  
            line_to_add = '' 
            line_to_add += str(i) + ': '
            line_to_add += str(list(optimized_params)) + '\n' + formatted_time + '\n'
            with open(file_path, 'a') as file:
                file.write(line_to_add)

## train per-subject 0.25 interp

In [ ]:
import pandas as pd
data = pd.read_csv('perez-fornos-2012.csv')  

# fixed parameters - condition dependent
class Args():
    EAR = 20 # electrode activation rate 
    exper_dur = 30 # experiment duration
    stim_dur = 10 # stimulus duration
    interpolation_factor = 2 # number of frames sampled every second
    
args = Args()

In [ ]:
# fit parameters
PER=2 # persistence duration (usually between 0 and 8 seconds)
PFO=10  # perceptual fading onset (usually between 0 and 10 seconds)
PFD=10 # perceptual fading duration (usually between 0.5 and 60 seconds)
brightness_scale = 10

def brightness_equ(PER, PFO, PFD, brightness_scale):
    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    ElectrodeArraySize=1 # number of electrodes (60(val=1), 256(2), 961(3), or 2916(4) electrodes)
    ElectrodeArrayStructure=1 # electrode array structure (square(val = 1) or hexagonal(2)).

    va = 0
    vb = 0.333
    vc = 0.667
    vd = 1

    SigmaX=.4 # phosphene dimension in x axis
    SigmaY=.4 # phosphene dimension in y axis
    MMr = 2  # number of rows
    MMc = 3  # number of columns;
    X0 = 20  # distance between two adjecent phosphenes in x axis
    Y0 = 20  # distance between two adjecent phosphenes in y axis
    C = MMc*Y0  # number of pixel columns
    R = MMr*X0  # number of pixel rows, choose: phosphene size 0.4; X0 and Y0 = 24

    DFR = 30 # display frame rate (usually 30 or 60 FPS)

    v2 = 0
    v3 = 0
    count = 0

    EAR = args.EAR  # electrode activation rate (usually 6 or 10 Hz)
    exper_dur = args.exper_dur  # experiment duration
    stim_dur = args.stim_dur  # stimulus duration

    PER_Factor = 3.9/(PER*DFR) # 3.9 was set for 2 precent threshold, i.e. the phosphene falls to 2 precent of its original intensity;
    PF_Factor = PFO*EAR
    PFD_Factor = 3.9/(PFD*DFR)

    # Creating meshgrid
    x = np.arange(0, C) 
    y = np.arange(0, R)  
    X, Y = np.meshgrid(x, y)
    # Initialization
    z = np.zeros((R, C))
    Frame = np.zeros((R, C))
    FormerFrame = np.zeros((R, C))
    PFC = np.zeros((MMr, MMc))
    Theta = np.zeros((MMr, MMc))
    a = np.zeros((MMr, MMc))
    b = np.zeros((MMr, MMc))
    c = np.zeros((MMr, MMc))
    sigma_x = np.zeros((MMr, MMc))
    sigma_y = np.zeros((MMr, MMc))

    count = 0
    for s in range(MMr):
        for t in range(MMc):
            Theta[s, t] = 0
            sigma_x[s, t] = SigmaX
            sigma_y[s, t] = SigmaY
            a[s, t] = (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)
            b[s, t] = -(np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_x[s, t]**2) + \
                      (np.sin(np.deg2rad(2 * Theta[s, t]))) / (4 * sigma_y[s, t]**2)
            c[s, t] = (np.sin(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_x[s, t]**2) + \
                      (np.cos(np.deg2rad(Theta[s, t]))**2) / (2 * sigma_y[s, t]**2)

    I_frame = [1]*(args.stim_dur*args.EAR)*MMr*MMc  # the brightness of the video is always 1
    # video frame rate is the same as the electrode firing rate
    
    I_frame_0 = [0]*(args.exper_dur*args.EAR-args.stim_dur*args.EAR)*MMr*MMc  # no visual input after stim_dur
    I_frame = np.append(np.array(I_frame), np.array(I_frame_0))
    I_frame = np.array(I_frame).reshape(args.exper_dur*args.EAR, MMr, MMc)

    results = []

    for frame in range(exper_dur*args.EAR):
        I = I_frame[frame]
        # I = I / I.max()
        MeanMatrix = np.zeros((MMr, MMc))
        s = 0
        t = 0
        v = 0

        start = int(np.floor((I.shape[0] % MMr) / 2) + np.floor(I.shape[0] / MMr) * MMr) - 1
        stop = int(1 + np.floor((I.shape[0] % MMr) / 2)) - 1
        step = -int(np.floor(I.shape[0] / MMr))
        for m in range(start, stop, step):
            for n in range(0, I.shape[1]//MMc*MMc - 1, I.shape[1] // MMc):
                for j in range(m, m - (I.shape[0] // MMr), -1):
                    # for k in range(n, n + (I.shape[1] // MMc)-2):
                    for k in range(n, n - (I.shape[1] // MMc), -1):
                        v += I[j, k]
                v /= ((I.shape[0] // MMr) * (I.shape[1] // MMc))

                if 0 <= v <= 0.25:
                    v = va
                elif 0.25 < v <= 0.5:
                    v = vb
                elif 0.5 < v <= 0.75:
                    v = vc
                else:
                    v = vd
                MeanMatrix[s, t] = v
                v = 0
                t += 1
            t = 0
            s += 1

        getting_all_subframe = []

        SubFrame_range = int(DFR) // int(EAR) + 1
        if SubFrame_range <= 2: SubFrame_range = 2
        for SubFrame in range(1, SubFrame_range):
            for s in range(MMr):
                for t in range(MMc):
                    adjusted_t = t + 1
                    adjusted_s = s + 1
                    z += MeanMatrix[s, t] * np.exp(
                        # -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        -0.015 * (1 / (3 * MeanMatrix[s, t] + 1)) *
                        # (a[s, t] * (X - X0 * t - v2 + X0 // 2) ** 2 +
                        (a[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) ** 2 +
                        # 2 * b[s, t] * (X - X0 * t - v2 + X0 // 2) * (Y - Y0 * s + Y0 // 2) +
                        2 * b[s, t] * (X - X0 * adjusted_t - v2 + X0 // 2) * (Y - Y0 * adjusted_s + Y0 // 2) +
                        # c[s, t] * (Y - Y0 * s + Y0 // 2) ** 2))
                        c[s, t] * (Y - Y0 * adjusted_s + Y0 // 2) ** 2))

                v2 -= (X0 / 2) * v3
                v3 *= -1

            for s in range(R, 0, -X0):
                for t in range(0, C, Y0):
                    if SubFrame == 1:
                        condition1 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) == 0 and np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) == 0)
                        condition2 = (np.mean(z[s-X0:s-1, t-1:t+Y0-2]) < np.mean(FormerFrame[s-X0:s-1, t:t+Y0-2]))
                        condition3 = (MeanMatrix[int(s/X0)-1, t // Y0] == 0)
                        condition4 = (PFC[int(s/X0)-1, t // Y0] < PF_Factor)
                        if condition1 or condition2 or condition3 and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                        elif np.mean(z[s-X0:s-1, t-1:t+Y0-2]) >= np.mean(FormerFrame[s-X0:s-1, t-1:t+Y0-2]) and condition4:
                            Frame[s-X0:s-1, t-1:t+Y0-2] = z[s-X0:s-1, t-1:t+Y0-2]
                            PFC[s//X0-1, t//Y0] += 1
                        elif PFC[s//X0-1, t//Y0] >= PF_Factor:
                            # Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-3 * PER_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                            Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PFD_Factor) * FormerFrame[s-X0:s-1, t-1:t+Y0-2]
                    else:
                        Frame[s-X0:s-1, t-1:t+Y0-2] = np.exp(-PER_Factor) * Frame[s-X0:s-1, t-1:t+Y0-2]
            z = np.zeros((R,C))
            count = count + 1

            FormerFrame = Frame
            getting_all_subframe.append(Frame.max())
        results.append(max(getting_all_subframe))

    output =  [x * brightness_scale for x in results]

    output_ = []
    combined_frame = args.EAR/args.interpolation_factor
    for i in range(args.exper_dur*args.interpolation_factor):
        output_.append(max(output[int(i*combined_frame):int((i+1)*combined_frame)]))
    return output_

In [ ]:
stim_info = {}
# EAR, exper_dur, stim_dur
for i in range(9): stim_info[i] = [20, 30, 10]
for i in range(9,18): stim_info[i] = [5, 30, 10]
for i in range(18,27): stim_info[i] = [60, 30, 10]
for i in range(27,36): stim_info[i] = [20, 10, 1]
for i in range(36,45): stim_info[i] = [20, 70, 60]

In [ ]:
import numpy as np
from scipy.optimize import minimize

args.interpolation_factor = 4

def objective_function(params, gt_curves_train, stim_lst):
    # persistence time, fading time onset, fading time duration, brightness scale
    PER, PFO, PFD, brightness_scale = params
    y_predicted = brightness_equ(PER, PFO, PFD, brightness_scale)
    
    
    total_mse = 0
    for gt_idx in range(len(gt_curves_train)):
        gt_y = gt_curves_train[gt_idx]
        # stim = stim_lst[gt_idx]
        # args.EAR = stim[0]
        # args.exper_dur = stim[1]
        # args.stim_dur = stim[2]

        total_mse += np.mean((np.array(y_predicted) - gt_y) ** 2)
    # print(total_mse/len(gt_curves_train))
    return total_mse/len(gt_curves_train)

for subj in range(9):
    for cond in range(5):

        i = cond*9 + subj
        print(i)

        if i in [0,9,18,27] : pass
        else:
            idx_lst = [cond*9 + sub for sub in range(9)]
            idx_lst.remove(i)
            
            args.EAR = stim_info[i][0]
            args.exper_dur = stim_info[i][1]
            args.stim_dur = stim_info[i][2]

            gt_curves_train = [np.nan_to_num(data.iloc[idx, 6:stim_info[idx][1]*args.interpolation_factor+6].to_numpy().astype('float64'), 0) for idx in idx_lst] 
            gt_curves_val = np.nan_to_num(data.iloc[i, 6:args.exper_dur*args.interpolation_factor+6].to_numpy().astype('float64'), 0)  

            stim_lst = [stim_info[idx] for idx in idx_lst]
            
            # Initial guesses for parameters
            initial_params = [2, 10, 10, 10]
            # Run the optimization, passing the ground truth as an argument
            result = minimize(objective_function, initial_params, args=(gt_curves_train,stim_lst), method='Nelder-Mead')
            # Extract optimized parameters
            optimized_params = result.x
            print("Optimized Parameters:", optimized_params)

            current_time = datetime.now()
            formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
            
            file_path = 'baseline_persubject_025interp.txt'  
            line_to_add = '' 
            line_to_add += str(i) + ': '
            line_to_add += str(list(optimized_params)) + '\n' + formatted_time + '\n'
            with open(file_path, 'a') as file:
                file.write(line_to_add)